# 📘 Introducción

## ¿Qué es el polipropileno?

El **polipropileno (PP)** es un polímero termoplástico perteneciente a la familia de las poliolefinas. Se obtiene mediante la **polimerización del propileno**, un derivado del petróleo.  
Es uno de los plásticos más utilizados a nivel mundial debido a su bajo costo, versatilidad, ligereza y resistencia química.

Entre sus principales aplicaciones destacan:

- Empaques y envases  
- Industria automotriz  
- Textiles y fibras  
- Construcción  
- Dispositivos médicos  
- Productos del hogar  

Su combinación de precio accesible, resistencia y facilidad de moldeo lo convierten en un material indispensable en la manufactura moderna.

---

## ¿Por qué su precio es relevante?

El precio del polipropileno es un indicador económico importante, pues está influenciado por factores interconectados como:

- Precios del petróleo y gas natural  
- Oferta y demanda global  
- Situación de la industria petroquímica  
- Políticas comerciales y condiciones macroeconómicas  
- Paros técnicos o fallas en plantas de producción  

Su relevancia se debe a que el PP afecta directamente los **costos de producción** de múltiples industrias.  
Por ello, contar con **predicciones confiables del precio del PP** ayuda a:

- Planificar compras y presupuestos  
- Evaluar riesgos de precios  
- Estimar márgenes  
- Realizar análisis económicos de mercado  

---

## Breve revisión del dataset

Los datos fueron obtenidos mediante un método de scraping controlado desde TradingEconomics, tomando los valores históricos del precio spot del polipropileno.

El dataset contiene dos columnas principales:

- **Date** — Fecha de registro  
- **Price** — Precio del polipropileno por tonelada  

Ejemplo de datos:

| Date       | Price |
|------------|-------|
| 2024-11-20 | 7489  |
| 2024-11-21 | 7510  |
| 2024-11-22 | 7504  |
| 2024-11-25 | 7467  |
| 2024-11-26 | 7479  |

Características del dataset:

- Frecuencia: **Diaria**  
- Formato: **serie temporal**  
- Moneda: **USD por tonelada**  
- Datos limpios y sin valores nulos  
- Ordenados de forma cronológica  



---


In [349]:
#importar datos desde tampermokey
import pandas as pd
import requests

df = pd.DataFrame(requests.get("http://localhost:8765/pp").json())

df = df.rename(columns={"date": "Date", "value": "Price"})
df["Date"] = pd.to_datetime(df["Date"])
df.head()


,Date,Price
0,2024-11-22,7504
1,2024-11-25,7467
2,2024-11-26,7479
3,2024-11-27,7481
4,2024-11-28,7482


In [351]:
import pandas as pd
from datetime import timedelta

# --- Tu código de carga y preparación ---
df = pd.read_csv("commodity_data.csv")
df = df.rename(columns={"date": "Date", "value": "Price"})
df['Date'] = pd.to_datetime(df['Date'])
df = df.set_index('Date')  # <--- Aquí la fecha se vuelve índice
df = df.asfreq('D')
df['Price'] = df['Price'].ffill()

# --- FILTRO DE 1 AÑO (CORREGIDO) ---
# Usamos df.index porque la fecha ya no es columna
fecha_final = df.index.max() 

# Calculamos fecha de inicio
fecha_inicio = fecha_final - timedelta(days=365)

# Filtramos usando el índice
df = df[df.index >= fecha_inicio]

print(f"📅 Datos reducidos al último año.")
print(f"Desde: {df.index.min().date()} Hasta: {df.index.max().date()}")
print(f"Cantidad de datos restantes: {len(df)}")

📅 Datos reducidos al último año.
Desde: 2024-11-21 Hasta: 2025-11-20
Cantidad de datos restantes: 365


In [352]:
import plotly.graph_objects as go

# Crear la figura
fig = go.Figure()

# Graficar los datos históricos
# Nota: Usamos df.index en 'x' porque hiciste set_index('Date')
fig.add_trace(go.Scatter(
    x=df.index,
    y=df['Price'],
    mode='lines',
    name='Precio Histórico',
    line=dict(color='#00CC96', width=2) # Color verde cian
))



# Configuración del diseño (Layout)
fig.update_layout(
    title="Precio del Polipropileno (PP) — Histórico",
    xaxis_title="Fecha",
    yaxis_title="Precio (CNY/T)",
    template="plotly_dark",      # Fondo oscuro
    hovermode="x unified",       # Muestra el valor al pasar el ratón
    legend=dict(
        orientation="h",         # Leyenda horizontal arriba
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
    )
)

fig.show()

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

In [353]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
df["Price_scaled"] = scaler.fit_transform(df[["Price"]])
df.head()


,Price,Price_scaled
Date,,
2024-11-21,7510.0,1.000000
2024-11-22,7504.0,0.994614
2024-11-23,7504.0,0.994614
2024-11-24,7504.0,0.994614
2024-11-25,7467.0,0.961400


In [354]:
import numpy as np

data = df["Price_scaled"].values

train_size = int(len(data) * 0.8)
train_data = data[:train_size]
test_data  = data[train_size:]


In [355]:
def create_windows(series, window_size=30):
    X, y = [], []
    for i in range(len(series) - window_size):
        X.append(series[i:i + window_size])
        y.append(series[i + window_size])
    return np.array(X), np.array(y)


In [356]:
WINDOW = 30  # puedes cambiarlo después

X_train, y_train = create_windows(train_data, WINDOW)
X_test,  y_test  = create_windows(test_data,  WINDOW)

# Ajustar dimensiones para LSTM/CNN
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test  = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

X_train.shape, X_test.shape


((262, 30, 1), (43, 30, 1))

# LTSM



In [357]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
# 1. Importar EarlyStopping
from tensorflow.keras.callbacks import EarlyStopping

model1 = Sequential([
    # Asegúrate de que 'WINDOW' esté definido previamente en tu código
    LSTM(64, return_sequences=True, input_shape=(WINDOW, 1)),
    Dropout(0.2),
    LSTM(32),
    Dropout(0.2),
    Dense(1)
])

model1.compile(optimizer="adam", loss="mse")
model1.summary()

# # # 2. Configurar el Early Stopping
# early_stop = EarlyStopping(
#     monitor='val_loss',       
#     patience=10,             
#     restore_best_weights=True 
# )

# 3. Añadirlo al entrenamiento con 'callbacks'
history = model1.fit(
    X_train, y_train,
    epochs=1000,               
    batch_size=16,
    validation_data=(X_test, y_test),
    # callbacks=[early_stop],   
    verbose=1
)

/Users/maxaguayo/Documents/modelos-no-lineales/cnn-Ltsm_prediccion_Polipropileno/.venv/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



Model: "sequential_53"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_54 (LSTM)                  │ (None, 30, 64)         │        16,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_80 (Dropout)            │ (None, 30, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_55 (LSTM)                  │ (None, 32)             │        12,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_81 (Dropout)            │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_79 (Dense)                │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 29,345 (114.63 KB)

 Trainable params: 29,345 (114.63 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/1000
17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - loss: 0.1037 - val_loss: 0.0182
Epoch 2/1000
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0199 - val_loss: 0.0479
Epoch 3/1000
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0117 - val_loss: 0.0318
Epoch 4/1000
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0098 - val_loss: 0.0322
Epoch 5/1000
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0104 - val_loss: 0.0287
Epoch 6/1000
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0097 - val_loss: 0.0251
Epoch 7/1000
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0097 - val_loss: 0.0235
Epoch 8/1000
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0074 - val_loss: 0.0185
Epoch 9/1000
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0087 - val_loss: 0.0232
Epoch 10/1000
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0082 - val_loss: 0.0185
Epoch 11/1000
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0091 - val_loss: 0.0133
Epoch 12/1000
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/ste

In [358]:
from tensorflow.keras.models import Sequential
# Aquí añadimos 'Input' que era lo que faltaba, además de las capas de CNN
from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D, GlobalAveragePooling1D, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# --- MODELO CNN 1D ---
model2 = Sequential([
    # Definimos la entrada explícitamente
    Input(shape=(X_train.shape[1], X_train.shape[2])), 
    
    # 1. Capa Convolucional: Busca patrones locales
    Conv1D(filters=64, kernel_size=3, activation='relu', padding='same'),
    
    # 2. Capa de Pooling: Reduce dimensionalidad
    MaxPooling1D(pool_size=2),
    
    # 3. Segunda Convolución
    Conv1D(filters=32, kernel_size=3, activation='relu', padding='same'),
    
    # 4. GlobalAveragePooling en lugar de Flatten (mejor para evitar overfitting)
    GlobalAveragePooling1D(), 
    
    Dropout(0.3),
    
    # 5. Capas densas finales
    Dense(16, activation='relu'),
    Dense(1)
])

model2.compile(optimizer='adam', loss='huber')

# Callbacks
# early_stop = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)
# reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6)

print("Entrenando CNN...")
history = model2.fit(
    X_train, y_train,
    epochs=500, 
    batch_size=8,
    validation_data=(X_test, y_test),
    # callbacks=[early_stop, reduce_lr],
    verbose=1
)

Entrenando CNN...
Epoch 1/500
33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0873 - val_loss: 0.0333
Epoch 2/500
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0161 - val_loss: 0.0254
Epoch 3/500
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0108 - val_loss: 0.0238
Epoch 4/500
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0075 - val_loss: 0.0286
Epoch 5/500
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0079 - val_loss: 0.0265
Epoch 6/500
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0070 - val_loss: 0.0250
Epoch 7/500
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0063 - val_loss: 0.0219
Epoch 8/500
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0062 - val_loss: 0.0259
Epoch 9/500
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0061 - val_loss: 0.0222
Epoch 10/500
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0049 - val_loss: 0.0235
Epoch 11/500
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0061 - val_loss: 0.0244
Epoch 12/500
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 1m

In [369]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go

# --- 1. GENERAR PREDICCIONES ---
# Predecimos sobre ambos conjuntos
train_pred_scaled = model1.predict(X_train, verbose=0)
test_pred_scaled = model1.predict(X_test, verbose=0)

# --- 2. INVERTIR ESCALA (Soporte para Univariante y Multivariante) ---
def inverse_transform_robust(pred_scaled, scaler):
    # Si el scaler espera más de 1 columna, creamos una matriz dummy
    if scaler.n_features_in_ > 1:
        dummy = np.zeros((len(pred_scaled), scaler.n_features_in_))
        dummy[:, 0] = pred_scaled.flatten()
        return scaler.inverse_transform(dummy)[:, 0]
    else:
        return scaler.inverse_transform(pred_scaled).flatten()

train_pred = inverse_transform_robust(train_pred_scaled, scaler)
test_pred = inverse_transform_robust(test_pred_scaled, scaler)

# --- 3. ALINEAR FECHAS ---
# El modelo empieza a predecir después de la primera ventana
# window_size se definió antes (ej. 14 o 20)
split_idx = len(train_pred)
total_offset = WINDOW

# Fechas para Train: Desde el final de la 1ra ventana hasta el corte
train_dates = df.index[total_offset : total_offset + split_idx]

# Fechas para Test: Desde el corte hasta el final
test_dates = df.index[total_offset + split_idx : total_offset + split_idx + len(test_pred)]

# --- 4. GRAFICAR ---
fig = go.Figure()

# A) Datos Originales (Fondo)
fig.add_trace(go.Scatter(
    x=df.index,
    y=df['Price'],
    mode='lines',
    name='Original (Histórico)',
    line=dict(color='gray', width=1, dash='solid'),
    opacity=0.5
))

# B) Predicción en Train (Azul)
fig.add_trace(go.Scatter(
    x=train_dates,
    y=train_pred,
    mode='lines',
    name='Predicción Train (Aprendizaje)',
    line=dict(color='#00CC96', width=2) # Verde/Cian
))

# C) Predicción en Test (Rojo/Naranja) -> LA IMPORTANTE
fig.add_trace(go.Scatter(
    x=test_dates,
    y=test_pred,
    mode='lines',
    name='Predicción Test (Validación)',
    line=dict(color='#EF553B', width=2) # Rojo
))

fig.update_layout(
    title="Evaluación del Modelo: Train vs Test vs Original",
    xaxis_title="Fecha",
    yaxis_title="Precio",
    template="plotly_dark",
    hovermode="x unified",
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
    )
)



ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'line': {'color': 'gray', 'dash': 'solid', 'width': 1},
              'mode': 'lines',
              'name': 'Original (Histórico)',
              'opacity': 0.5,
              'type': 'scatter',
              'x': array(['2024-11-21T00:00:00.000000000', '2024-11-22T00:00:00.000000000',
                          '2024-11-23T00:00:00.000000000', ..., '2025-11-18T00:00:00.000000000',
                          '2025-11-19T00:00:00.000000000', '2025-11-20T00:00:00.000000000'],
                         shape=(365,), dtype='datetime64[ns]'),
              'y': {'bdata': ('AAAAAABWvUAAAAAAAFC9QAAAAAAAUL' ... 'AAG7lAAAAAAAAWuUAAAAAAAPy4QA=='),
                    'dtype': 'f8'}},
             {'line': {'color': '#00CC96', 'width': 2},
              'mode': 'lines',
              'name': 'Predicción Train (Aprendizaje)',
              'type': 'scatter',
              'x': array(['2024-12-21T00:00:00.000000000', '2024-12-22T00:00:00.000000000',
                          '2024-12-23T00:00:00.000000000', ..., '2025-09-06T00:00:00.000000000',
                          '2025-09-07T00:00:00.000000000', '2025-09-08T00:00:00.000000000'],
                         shape=(262,), dtype='datetime64[ns]'),
              'y': {'bdata': ('9dDmRUXJ5kUQ2OZFjCbnReT450XY++' ... 'WZeddFdgDWRcIl10UUgtZFRavWRQ=='),
                    'dtype': 'f4'}},
             {'line': {'color': '#EF553B', 'width': 2},
              'mode': 'lines',
              'name': 'Predicción Test (Validación)',
              'type': 'scatter',
              'x': array(['2025-09-09T00:00:00.000000000', '2025-09-10T00:00:00.000000000',
                          '2025-09-11T00:00:00.000000000', '2025-09-12T00:00:00.000000000',
                          '2025-09-13T00:00:00.000000000', '2025-09-14T00:00:00.000000000',
                          '2025-09-15T00:00:00.000000000', '2025-09-16T00:00:00.000000000',
                          '2025-09-17T00:00:00.000000000', '2025-09-18T00:00:00.000000000',
                          '2025-09-19T00:00:00.000000000', '2025-09-20T00:00:00.000000000',
                          '2025-09-21T00:00:00.000000000', '2025-09-22T00:00:00.000000000',
                          '2025-09-23T00:00:00.000000000', '2025-09-24T00:00:00.000000000',
                          '2025-09-25T00:00:00.000000000', '2025-09-26T00:00:00.000000000',
                          '2025-09-27T00:00:00.000000000', '2025-09-28T00:00:00.000000000',
                          '2025-09-29T00:00:00.000000000', '2025-09-30T00:00:00.000000000',
                          '2025-10-01T00:00:00.000000000', '2025-10-02T00:00:00.000000000',
                          '2025-10-03T00:00:00.000000000', '2025-10-04T00:00:00.000000000',
                          '2025-10-05T00:00:00.000000000', '2025-10-06T00:00:00.000000000',
                          '2025-10-07T00:00:00.000000000', '2025-10-08T00:00:00.000000000',
                          '2025-10-09T00:00:00.000000000', '2025-10-10T00:00:00.000000000',
                          '2025-10-11T00:00:00.000000000', '2025-10-12T00:00:00.000000000',
                          '2025-10-13T00:00:00.000000000', '2025-10-14T00:00:00.000000000',
                          '2025-10-15T00:00:00.000000000', '2025-10-16T00:00:00.000000000',
                          '2025-10-17T00:00:00.000000000', '2025-10-18T00:00:00.000000000',
                          '2025-10-19T00:00:00.000000000', '2025-10-20T00:00:00.000000000',
                          '2025-10-21T00:00:00.000000000'], dtype='datetime64[ns]'),
              'y': {'bdata': ('gHnURS+g0EUmkNFF7M3PRRXDz0Vkvs' ... 'VPWMlF57HKRfnoyUU33clF9p3JRQ=='),
                    'dtype': 'f4'}}],
    'layout': {'hovermode': 'x unified',
               'legend': {'orientation': 'h', 'x': 1, 'xanchor': 'right', 'y': 1.02, 'yanchor': 'bottom'},
               'template': '...',
               'title': {'text': 'Evaluación del Modelo: Train vs Test vs Original'},
               'xaxis': {'title': {'

## MSE y MAPE

In [360]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error

# --- Función para evaluar manejando el des-escalado ---
def evaluar_modelo(model, X_test, y_test, scaler, nombre):
    # 1. Predecir
    # El modelo devuelve valores entre 0 y 1
    pred_scaled = model.predict(X_test, verbose=0)
    
    # 2. Preparar matrices para invertir la escala (Inverse Transform)
    # El scaler espera (filas, n_features), así que creamos una matriz 'dummy'
    # y rellenamos solo la primera columna con nuestros datos de precio.
    n_features = scaler.n_features_in_
    
    # -- Des-escalar Predicción --
    dummy_pred = np.zeros((len(pred_scaled), n_features))
    dummy_pred[:, 0] = pred_scaled.flatten()
    y_pred_real = scaler.inverse_transform(dummy_pred)[:, 0]
    
    # -- Des-escalar Real (y_test) --
    dummy_true = np.zeros((len(y_test), n_features))
    dummy_true[:, 0] = y_test.flatten()
    y_true_real = scaler.inverse_transform(dummy_true)[:, 0]
    
    # 3. Calcular Métricas
    mse = mean_squared_error(y_true_real, y_pred_real)
    mape = mean_absolute_percentage_error(y_true_real, y_pred_real) * 100
    
    return mse, mape, y_pred_real, y_true_real

# --- EJECUCIÓN PARA TUS DOS MODELOS ---

print("📊 Calculando métricas...")

# 1. Evaluar LSTM (model1)
mse1, mape1, pred1, y_real = evaluar_modelo(model1, X_test, y_test, scaler, "LSTM")

# 2. Evaluar CNN (model2)
# Usamos el mismo y_real devuelto arriba, no hace falta recalcularlo
mse2, mape2, pred2, _ = evaluar_modelo(model2, X_test, y_test, scaler, "CNN")

# --- MOSTRAR RESULTADOS ---

print("\n" + "="*40)
print(f"🏆 RESULTADOS FINALES")
print("="*40)
print(f"🔹 Model 1 (LSTM):")
print(f"   MSE:  {mse1:.2f}")
print(f"   MAPE: {mape1:.4f}%  <-- Error porcentual promedio")
print("-" * 40)
print(f"🔸 Model 2 (CNN):")
print(f"   MSE:  {mse2:.2f}")
print(f"   MAPE: {mape2:.4f}%  <-- Error porcentual promedio")
print("="*40)

# Decidir ganador
ganador = "LSTM (Model 1)" if mape1 < mape2 else "CNN (Model 2)"
diferencia = abs(mape1 - mape2)
print(f"✅ El ganador es: {ganador} (Mejor por {diferencia:.4f}%)")




📊 Calculando métricas...

🏆 RESULTADOS FINALES
🔹 Model 1 (LSTM):
   MSE:  2087.64
   MAPE: 0.5424%  <-- Error porcentual promedio
----------------------------------------
🔸 Model 2 (CNN):
   MSE:  65859.72
   MAPE: 3.7462%  <-- Error porcentual promedio
✅ El ganador es: LSTM (Model 1) (Mejor por 3.2037%)


## Predicciones 7 días

In [362]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# --- 1. CONFIGURACIÓN ---
n_future = 7
features = ['Price'] # Solo estamos usando Price

# Detectar automáticamente si Date es columna o índice
if "Date" in df.columns:
    last_date = df["Date"].iloc[-1]
else:
    last_date = df.index[-1]

future_dates = pd.date_range(last_date, periods=n_future+1, freq="D")[1:]

# --- 2. PREPARAR LA ÚLTIMA VENTANA (Univariante) ---
# Tomamos solo la columna Price
last_window_data = df[features].tail(WINDOW).values 
# Escalamos (asegúrate de que 'scaler' fue entrenado solo con Price también)
last_window_scaled = scaler.transform(last_window_data)

# --- 3. FUNCIÓN DE PREDICCIÓN (Lógica Simplificada) ---
def predecir_univariante(model, initial_window, steps):
    # Damos forma (1, window_size, 1)
    current_batch = initial_window.reshape(1, WINDOW, 1)
    predictions = []
    
    for _ in range(steps):
        # Predecir el siguiente punto
        pred_val = model.predict(current_batch, verbose=0)[0][0]
        predictions.append(pred_val)
        
        # Actualizar ventana: Quitamos el primero y añadimos la predicción al final
        # En univariante es simple: solo añadimos el valor predicho
        new_entry = np.array([[[pred_val]]]) 
        current_batch = np.append(current_batch[:, 1:, :], new_entry, axis=1)
        
    return predictions

# --- 4. EJECUTAR PREDICCIONES ---
print("🔮 Generando predicciones futuras (Solo Precio)...")

# Predicciones escaladas
preds_1 = predecir_univariante(model1, last_window_scaled, n_future) # LSTM
preds_2 = predecir_univariante(model2, last_window_scaled, n_future) # CNN

# --- 5. INVERTIR ESCALA ---
# Como el scaler es univariante, es directo
future_real_1 = scaler.inverse_transform(np.array(preds_1).reshape(-1, 1)).flatten()
future_real_2 = scaler.inverse_transform(np.array(preds_2).reshape(-1, 1)).flatten()

# --- 6. RESULTADOS Y GRÁFICO ---
df_futuro = pd.DataFrame({
    "Fecha": future_dates,
    "LSTM ($)": future_real_1,
    "CNN ($)": future_real_2,
    "Promedio": (future_real_1 + future_real_2) / 2
})

print("\n--- Pronóstico a 7 días ---")
print(df_futuro)



/Users/maxaguayo/Documents/modelos-no-lineales/cnn-Ltsm_prediccion_Polipropileno/.venv/lib/python3.12/site-packages/sklearn/utils/validation.py:2749: UserWarning:

X does not have valid feature names, but MinMaxScaler was fitted with feature names



🔮 Generando predicciones futuras (Solo Precio)...

--- Pronóstico a 7 días ---
       Fecha     LSTM ($)      CNN ($)     Promedio
0 2025-11-21  6426.715820  6759.407715  6593.061523
1 2025-11-22  6446.798340  6779.331543  6613.064941
2 2025-11-23  6436.644531  6770.036133  6603.340332
3 2025-11-24  6438.898926  6761.123535  6600.011230
4 2025-11-25  6442.436035  6764.038574  6603.237305
5 2025-11-26  6444.234863  6759.049805  6601.642578
6 2025-11-27  6447.746094  6766.159180  6606.952637


## Visualización próximas

In [364]:
import plotly.graph_objects as go

# Crear la figura base
fig = go.Figure()

# --- 1. DATOS HISTÓRICOS ---
# Mostramos solo los últimos 60 días para que el gráfico no se vea muy alejado
# (Si quieres ver todo, quita el .iloc[-60:])
datos_recientes = df.iloc[-60:]
fechas_historia = datos_recientes["Date"] if "Date" in datos_recientes.columns else datos_recientes.index

fig.add_trace(go.Scatter(
    x=fechas_historia,
    y=datos_recientes["Price"],
    mode="lines",
    name="Histórico",
    line=dict(color="#00CC96", width=2) # Color Cian/Verde
))

# --- 2. PREDICCIÓN LSTM (Model 1) ---
fig.add_trace(go.Scatter(
    x=future_dates,
    y=future_real_1,  # Variable de predicción LSTM
    mode="lines+markers",
    name="Predicción LSTM",
    line=dict(color="#AB63FA", dash="dash") # Morado, línea discontinua
))

# --- 3. PREDICCIÓN CNN (Model 2) ---
fig.add_trace(go.Scatter(
    x=future_dates,
    y=future_real_2,  # Variable de predicción CNN
    mode="lines+markers",
    name="Predicción CNN",
    line=dict(color="#EF553B", dash="dot") # Rojo, línea de puntos
))

# --- 4. PROMEDIO (Opcional) ---
# A veces el promedio es la apuesta más segura
promedio = (future_real_1 + future_real_2) / 2
fig.add_trace(go.Scatter(
    x=future_dates,
    y=promedio,
    mode="lines",
    name="Promedio (Ensemble)",
    line=dict(color="white", width=1),
    opacity=0.5
))

# --- CONFIGURACIÓN DE ESTILO (Template Dark) ---
fig.update_layout(
    title="Comparativa: Histórico vs Predicciones (7 días)",
    xaxis_title="Fecha",
    yaxis_title="Precio ($/ton)",
    template="plotly_dark",      # El tema oscuro que pediste
    hovermode="x unified",       # Muestra todos los valores al pasar el mouse por una fecha
    legend=dict(
        orientation="h",         # Leyenda horizontal
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
    )
)



ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'line': {'color': '#00CC96', 'width': 2},
              'mode': 'lines',
              'name': 'Histórico',
              'type': 'scatter',
              'x': array(['2025-09-22T00:00:00.000000000', '2025-09-23T00:00:00.000000000',
                          '2025-09-24T00:00:00.000000000', '2025-09-25T00:00:00.000000000',
                          '2025-09-26T00:00:00.000000000', '2025-09-27T00:00:00.000000000',
                          '2025-09-28T00:00:00.000000000', '2025-09-29T00:00:00.000000000',
                          '2025-09-30T00:00:00.000000000', '2025-10-01T00:00:00.000000000',
                          '2025-10-02T00:00:00.000000000', '2025-10-03T00:00:00.000000000',
                          '2025-10-04T00:00:00.000000000', '2025-10-05T00:00:00.000000000',
                          '2025-10-06T00:00:00.000000000', '2025-10-07T00:00:00.000000000',
                          '2025-10-08T00:00:00.000000000', '2025-10-09T00:00:00.000000000',
                          '2025-10-10T00:00:00.000000000', '2025-10-11T00:00:00.000000000',
                          '2025-10-12T00:00:00.000000000', '2025-10-13T00:00:00.000000000',
                          '2025-10-14T00:00:00.000000000', '2025-10-15T00:00:00.000000000',
                          '2025-10-16T00:00:00.000000000', '2025-10-17T00:00:00.000000000',
                          '2025-10-18T00:00:00.000000000', '2025-10-19T00:00:00.000000000',
                          '2025-10-20T00:00:00.000000000', '2025-10-21T00:00:00.000000000',
                          '2025-10-22T00:00:00.000000000', '2025-10-23T00:00:00.000000000',
                          '2025-10-24T00:00:00.000000000', '2025-10-25T00:00:00.000000000',
                          '2025-10-26T00:00:00.000000000', '2025-10-27T00:00:00.000000000',
                          '2025-10-28T00:00:00.000000000', '2025-10-29T00:00:00.000000000',
                          '2025-10-30T00:00:00.000000000', '2025-10-31T00:00:00.000000000',
                          '2025-11-01T00:00:00.000000000', '2025-11-02T00:00:00.000000000',
                          '2025-11-03T00:00:00.000000000', '2025-11-04T00:00:00.000000000',
                          '2025-11-05T00:00:00.000000000', '2025-11-06T00:00:00.000000000',
                          '2025-11-07T00:00:00.000000000', '2025-11-08T00:00:00.000000000',
                          '2025-11-09T00:00:00.000000000', '2025-11-10T00:00:00.000000000',
                          '2025-11-11T00:00:00.000000000', '2025-11-12T00:00:00.000000000',
                          '2025-11-13T00:00:00.000000000', '2025-11-14T00:00:00.000000000',
                          '2025-11-15T00:00:00.000000000', '2025-11-16T00:00:00.000000000',
                          '2025-11-17T00:00:00.000000000', '2025-11-18T00:00:00.000000000',
                          '2025-11-19T00:00:00.000000000', '2025-11-20T00:00:00.000000000'],
                         dtype='datetime64[ns]'),
              'y': {'bdata': ('AAAAAAClukAAAAAAAH66QAAAAAAAlr' ... 'AAAAAbuUAAAAAAABa5QAAAAAAA/LhA'),
                    'dtype': 'f8'}},
             {'line': {'color': '#AB63FA', 'dash': 'dash'},
              'mode': 'lines+markers',
              'name': 'Predicción LSTM',
              'type': 'scatter',
              'x': array(['2025-11-21T00:00:00.000000000', '2025-11-22T00:00:00.000000000',
                          '2025-11-23T00:00:00.000000000', '2025-11-24T00:00:00.000000000',
                          '2025-11-25T00:00:00.000000000', '2025-11-26T00:00:00.000000000',
                          '2025-11-27T00:00:00.000000000'], dtype='datetime64[ns]'),
              'y': {'bdata': 'utXIRWN2yUUoJclFMTfJRX1TyUXhYclF+H3JRQ==', 'dtype': 'f4'}},
             {'line': {'color': '#EF553B', 'dash': 'dot'},
              'mode': 'lines+markers',
              'name': 'Predicción CNN',
              'type': 'scatter',
              'x': array(['2025-11-21T00:00:00.000000000', '2025-11-22T00:00:00.000000000',

## Predicciones

In [365]:
# Crear la tabla comparativa con las variables que ya calculamos
future_table = pd.DataFrame({
    "Fecha futura": future_dates,
    "LSTM ($/ton)": future_real_1,
    "CNN ($/ton)": future_real_2,
    "Promedio ($/ton)": (future_real_1 + future_real_2) / 2
})

# Opcional: Redondear a 2 decimales para que se vea limpio
future_table = future_table.round(2)

# Mostrar la tabla
future_table

,Fecha futura,LSTM ($/ton),CNN ($/ton),Promedio ($/ton)
0,2025-11-21,6426.720215,6759.410156,6593.060059
1,2025-11-22,6446.799805,6779.330078,6613.060059
2,2025-11-23,6436.640137,6770.040039,6603.339844
3,2025-11-24,6438.899902,6761.120117,6600.009766
4,2025-11-25,6442.439941,6764.040039,6603.240234
5,2025-11-26,6444.240234,6759.049805,6601.640137
6,2025-11-27,6447.750000,6766.160156,6606.950195
